In [0]:
#load the data into databricks
df = spark.read.csv("/FileStore/tables/banking_transactions_data.csv", header=True, inferSchema=True)
df.show()


+--------------+-----------+----------+------+--------------------+---------------+----------------+----------------+------------------+
|TRANSACTION_ID|CUSTOMER_ID|DATE_BIRTH|GENDER|       CUST_LOCATION|ACCOUNT_BALANCE|TRANSACTION_DATE|TRANSACTION_TIME|TRANSACTION_AMOUNT|
+--------------+-----------+----------+------+--------------------+---------------+----------------+----------------+------------------+
|          T244|   C4531588|  93-10-31|     F|               DELHI|        2257.58|      2021-10-16|          145417|             460.0|
|          T245|   C5528968|  00-01-01|     M|            PIPARIYA|         5724.5|      2021-10-16|          145424|            4478.0|
|          T246|   C4814390|  71-10-01|     F|         MIRA ROAD E|       78122.46|      2021-10-16|          150436|           47290.0|
|          T247|   C6437438|  83-05-22|     F|        BHUBANESHWAR|       324551.5|      2021-10-16|          151154|           25000.0|
|          T248|   C8738676|  00-01-01|  

In [0]:
import pandas as pd
dfp = df.toPandas()

## Data validation
We will start by validating the data by analyzing the contents of the dataframe to compare to the same analysis done before the migration

In [0]:
#show number of entries (Expected = 250 000)
df.count()

250000

In [0]:
#show all features in the dataset (Expected = 9 features)
df.columns

['TRANSACTION_ID',
 'CUSTOMER_ID',
 'DATE_BIRTH',
 'GENDER',
 'CUST_LOCATION',
 'ACCOUNT_BALANCE',
 'TRANSACTION_DATE',
 'TRANSACTION_TIME',
 'TRANSACTION_AMOUNT']

In [0]:
# Show how many entries in top 5 cities
# Expected =
# MUMBAI = 24631
# BANGALORE = 20346
# NEW DELHI = 20090
# GURGAON = 17716
# DELHI = 16732)
df.groupby("CUST_LOCATION").count().orderBy("count", ascending=False).show(5)

+-------------+-----+
|CUST_LOCATION|count|
+-------------+-----+
|       MUMBAI|24631|
|    BANGALORE|20346|
|    NEW DELHI|20090|
|      GURGAON|17716|
|        DELHI|16732|
+-------------+-----+
only showing top 5 rows



In [0]:
# Average account balance (Expected = 109716.9143153)
dfp["ACCOUNT_BALANCE"].mean()

109716.9143153905

In [0]:
# Total amount spent (Expected = 400237664.81)
dfp["TRANSACTION_AMOUNT"].sum()

400237664.81

In [0]:
# how many male customers had an account balance of less than 2000 (Expected = 32477)
dfp.query("ACCOUNT_BALANCE < 2000 and GENDER == 'M'").shape[0]

32477

In [0]:
# What customer ID is associated with Transaction ID T246143 (Expected = C6562053)
dfp.query("TRANSACTION_ID == 'T246143'")["CUSTOMER_ID"]

249995    C6562053
Name: CUSTOMER_ID, dtype: object

## Store data in azure cloud
Next we will store the migrated data in a databricks cloud table.

In [0]:
# Store dataframe to a dtabricks delta table
df.write.format("delta").save("dbfs:/mnt/delta/banking_transaction")
